# Test for Issue #543 Solution: Annotated YAML File Generation

This notebook tests the complete solution that:
1. Fixes the misleading "will shortly be generated" message
2. Provides clear instructions with full file paths
3. Adds optional automatic generation via `auto_generate_annotated` parameter

In [ ]:
# Import required modules
import os
import sys
import tempfile
from pathlib import Path
import logging
from io import StringIO

# Add the src directory to Python path
sys.path.insert(0, str(Path('.').absolute() / 'src'))

# Import supy modules
import supy as sp
from supy.data_model.core import SUEWSConfig
from supy import SUEWSSimulation

print(f"SuPy version: {sp.__version__ if hasattr(sp, '__version__') else 'development'}")

## Test Configuration

Create a test YAML configuration with missing parameters that will trigger validation warnings.

In [ ]:
# Test YAML configuration with missing parameters
test_yaml_content = """# Test configuration for Issue #543
name: "Issue 543 Test Config"
description: "Testing annotated YAML generation behavior"

sites:
  - name: "Urban Test Site"
    gridiv: 1
    properties:
      lat: {value: 51.5074}
      lng: {value: -0.1278}
      land_cover:
        bldgs:
          sfr: {value: 0.45}  # 45% building coverage
          # MISSING: bldgh (building height)
          # MISSING: faibldg (frontal area index)
        paved:
          sfr: {value: 0.25}
        evergreen_tree:
          sfr: {value: 0.20}
        grass:
          sfr: {value: 0.10}

model:
  control:
    # Both syntaxes now work for tstep and diagnose!
    tstep: {value: 3600}    # RefValue syntax (now supported)
    diagnose: 0             # Direct value syntax (still works)
"""

print("Test YAML configuration created with:")
print("- 45% building coverage")
print("- Missing building height (bldgh)")
print("- Missing frontal area index (faibldg)")
print("\\nNote: tstep and diagnose now accept both direct values and {value: ...} syntax!")
print("This should trigger validation warnings.")

## Test 1: Default Behavior (Manual Generation)

Test the default behavior where annotated YAML is NOT automatically generated.

In [ ]:
print("TEST 1: Default behavior (auto_generate_annotated=False)")
print("=" * 60)

with tempfile.TemporaryDirectory() as temp_dir:
    temp_path = Path(temp_dir)
    yaml_file = temp_path / "test_default.yml"
    
    # Write test YAML
    yaml_file.write_text(test_yaml_content)
    print(f"\nCreated test YAML: {yaml_file}")
    
    # Set up logging capture
    log_capture = StringIO()
    handler = logging.StreamHandler(log_capture)
    handler.setLevel(logging.WARNING)
    logger = logging.getLogger("SuPy")
    logger.addHandler(handler)
    
    try:
        # Load configuration with default settings
        print("\nLoading configuration (default behavior)...")
        config = SUEWSConfig.from_yaml(str(yaml_file))
        
        # Get log output
        log_output = log_capture.getvalue()
        
        # Check validation message
        print("\nValidation Summary:")
        print("-" * 40)
        if "VALIDATION SUMMARY" in log_output:
            # Extract and print the validation summary
            summary_start = log_output.find("VALIDATION SUMMARY")
            summary_end = log_output.find("="*60, summary_start + 100)
            if summary_end > summary_start:
                print(log_output[summary_start:summary_end])
        
        # Check for instructions
        print("\nChecking instructions in message:")
        checks = [
            ("generate_annotated_yaml" in log_output, "Contains generate_annotated_yaml command"),
            (str(yaml_file) in log_output, "Shows full path to config file"),
            ("test_default_annotated.yml" in log_output, "Shows annotated file name"),
            ("Run: config." in log_output, "Has clear run instruction"),
            ("will shortly be generated" not in log_output, "Does NOT have misleading message")
        ]
        
        for check, description in checks:
            status = "✅" if check else "❌"
            print(f"  {status} {description}")
        
        # Check if file was created
        annotated_file = temp_path / "test_default_annotated.yml"
        print(f"\nChecking for auto-generated file: {annotated_file}")
        if annotated_file.exists():
            print("❌ UNEXPECTED: File was automatically generated (should not be)")
        else:
            print("✅ CORRECT: No file automatically generated (as expected)")
            
        # Now test manual generation
        print("\n" + "-"*40)
        print("Testing manual generation...")
        generated_path = config.generate_annotated_yaml(str(yaml_file))
        print(f"Generated: {generated_path}")
        
        if Path(generated_path).exists():
            print("✅ Manual generation successful")
            # Show first few lines
            content = Path(generated_path).read_text()
            lines = content.split('\n')[:15]
            print("\nFirst 15 lines of annotated file:")
            for i, line in enumerate(lines, 1):
                print(f"{i:3d}: {line}")
            Path(generated_path).unlink()  # Clean up
    
    finally:
        logger.removeHandler(handler)

## Test 2: Automatic Generation Enabled

Test with `auto_generate_annotated=True` to verify automatic file generation.

In [ ]:
print("TEST 2: Automatic generation (auto_generate_annotated=True)")
print("=" * 60)

with tempfile.TemporaryDirectory() as temp_dir:
    temp_path = Path(temp_dir)
    yaml_file = temp_path / "test_auto.yml"
    
    # Write test YAML
    yaml_file.write_text(test_yaml_content)
    print(f"\nCreated test YAML: {yaml_file}")
    
    # Set up logging capture
    log_capture = StringIO()
    handler = logging.StreamHandler(log_capture)
    handler.setLevel(logging.INFO)  # Capture INFO level to see generation message
    logger = logging.getLogger("SuPy")
    logger.addHandler(handler)
    
    try:
        # Load with automatic generation enabled
        print("\nLoading configuration with auto_generate_annotated=True...")
        config = SUEWSConfig.from_yaml(str(yaml_file), auto_generate_annotated=True)
        
        # Get log output
        log_output = log_capture.getvalue()
        
        # Check validation message
        print("\nChecking validation message:")
        checks = [
            ("will be generated" in log_output, "Says file 'will be generated'"),
            ("test_auto_annotated.yml" in log_output, "Shows annotated file name"),
            ("Run: config." not in log_output, "Does NOT show manual command"),
            ("Annotated YAML file generated:" in log_output, "Shows generation confirmation")
        ]
        
        for check, description in checks:
            status = "✅" if check else "❌"
            print(f"  {status} {description}")
        
        # Check if file was created
        annotated_file = temp_path / "test_auto_annotated.yml"
        print(f"\nChecking for auto-generated file: {annotated_file}")
        
        if annotated_file.exists():
            print("✅ SUCCESS: File was automatically generated!")
            print(f"   File size: {annotated_file.stat().st_size} bytes")
            
            # Verify content
            content = annotated_file.read_text()
            print("\nContent verification:")
            content_checks = [
                ("ANNOTATED SUEWS CONFIGURATION" in content, "Has header"),
                ("[ERROR] MISSING:" in content or "[WARNING] MISSING:" in content, "Has error markers"),
                ("bldgh" in content, "Notes missing building height"),
                ("faibldg" in content, "Notes missing frontal area"),
                ("[TIP]" in content, "Has fix tips")
            ]
            
            for check, description in content_checks:
                status = "✅" if check else "❌"
                print(f"  {status} {description}")
                
            # Show relevant sections
            print("\nRelevant annotated sections:")
            lines = content.split('\n')
            for i, line in enumerate(lines):
                if '[ERROR]' in line or '[WARNING]' in line or '[TIP]' in line or 'bldgh' in line:
                    print(f"  Line {i+1}: {line.strip()}")
                    if i < len(lines) - 1:
                        next_line = lines[i+1].strip()
                        if next_line and not next_line.startswith('#'):
                            print(f"  Line {i+2}: {next_line}")
        else:
            print("❌ FAILURE: File was NOT automatically generated!")
    
    finally:
        logger.removeHandler(handler)

## Test 3: No Validation Issues

Test that no annotated file is generated when there are no validation issues.

In [ ]:
print("TEST 3: No validation issues scenario")
print("=" * 60)

# Valid configuration with all required parameters
valid_yaml = """name: "Valid Test Config"
sites:
  - name: "Complete Site"
    gridiv: 1
    properties:
      lat: {value: 51.5}
      lng: {value: -0.1}
      land_cover:
        bldgs:
          sfr: {value: 0.3}
          bldgh: {value: 15.0}  # Building height provided
          faibldg: {value: 0.4}  # Frontal area provided
        paved:
          sfr: {value: 0.7}
model:
  control:
    tstep: 3600    # Plain int
"""

with tempfile.TemporaryDirectory() as temp_dir:
    temp_path = Path(temp_dir)
    yaml_file = temp_path / "test_valid.yml"
    yaml_file.write_text(valid_yaml)
    
    # Capture logs
    log_capture = StringIO()
    handler = logging.StreamHandler(log_capture)
    handler.setLevel(logging.WARNING)
    logger = logging.getLogger("SuPy")
    logger.addHandler(handler)
    
    try:
        # Load with auto_generate_annotated=True
        print("\nLoading valid configuration with auto_generate_annotated=True...")
        config = SUEWSConfig.from_yaml(str(yaml_file), auto_generate_annotated=True)
        
        log_output = log_capture.getvalue()
        
        # Check for validation warnings
        if "VALIDATION SUMMARY" in log_output:
            print("⚠️  Unexpected validation warnings found")
            print(log_output)
        else:
            print("✅ No validation warnings (as expected)")
        
        # Check if annotated file was created
        annotated_file = temp_path / "test_valid_annotated.yml"
        if annotated_file.exists():
            print("⚠️  Annotated file created even with no issues")
        else:
            print("✅ No annotated file created (correct behavior)")
    
    finally:
        logger.removeHandler(handler)

## Test 4: SUEWSSimulation Class Integration

Test how the solution works with the high-level SUEWSSimulation API.

In [ ]:
print("TEST 4: SUEWSSimulation class behavior")
print("=" * 60)

with tempfile.TemporaryDirectory() as temp_dir:
    temp_path = Path(temp_dir)
    yaml_file = temp_path / "test_simulation.yml"
    
    # Write test YAML
    yaml_file.write_text(test_yaml_content)
    print(f"\nCreated test YAML: {yaml_file}")
    
    # Capture logs
    log_capture = StringIO()
    handler = logging.StreamHandler(log_capture)
    handler.setLevel(logging.WARNING)
    logger = logging.getLogger("SuPy")
    logger.addHandler(handler)
    
    try:
        # Load via SUEWSSimulation (uses default behavior)
        print("\nLoading via SUEWSSimulation class...")
        sim = SUEWSSimulation(str(yaml_file))
        
        log_output = log_capture.getvalue()
        
        # Check message
        print("\nChecking behavior with SUEWSSimulation:")
        if "VALIDATION SUMMARY" in log_output:
            print("✅ Validation warnings shown")
            if "generate_annotated_yaml" in log_output:
                print("✅ Shows manual generation instructions")
            else:
                print("❌ Missing manual generation instructions")
        
        # Check no auto-generation
        annotated_file = temp_path / "test_simulation_annotated.yml"
        if not annotated_file.exists():
            print("✅ No automatic file generation (correct default)")
        else:
            print("❌ Unexpected automatic generation")
            
        # Manual generation through config object
        print("\nAccessing config object for manual generation...")
        if hasattr(sim, '_config') and sim._config:
            generated = sim._config.generate_annotated_yaml(str(yaml_file))
            print(f"✅ Manual generation successful: {generated}")
            Path(generated).unlink()  # Clean up
        else:
            print("⚠️  Cannot access config object from simulation")
    
    finally:
        logger.removeHandler(handler)

## Bonus: Flexible Syntax for tstep and diagnose

As part of this fix, we've also made `tstep` and `diagnose` accept both direct values and RefValue syntax for consistency with other fields.

In [ ]:
print("BONUS: Testing flexible syntax for tstep and diagnose")
print("=" * 60)

# Test YAML with different syntax combinations
flexible_yaml = """name: "Flexible Syntax Test"
sites:
  - name: "Test Site"
    gridiv: 1
    properties:
      lat: {value: 51.5}
      lng: {value: -0.1}
model:
  control:
    # Example 1: Direct values (original syntax)
    tstep: 300
    diagnose: 0
    
    # Example 2: RefValue syntax (now also supported!)
    # tstep: {value: 300}
    # diagnose: {value: 0}
    
    # Example 3: RefValue with references
    # tstep: 
    #   value: 300
    #   ref:
    #     desc: "5-minute timestep for detailed analysis"
    # diagnose:
    #   value: 2
    #   ref:
    #     desc: "Full diagnostics for debugging"
"""

with tempfile.TemporaryDirectory() as temp_dir:
    temp_path = Path(temp_dir)
    yaml_file = temp_path / "flexible_test.yml"
    yaml_file.write_text(flexible_yaml)
    
    try:
        print("\nLoading configuration with direct values...")
        config = SUEWSConfig.from_yaml(str(yaml_file))
        print(f"✅ tstep loaded: {config.model.control.tstep}")
        print(f"✅ diagnose loaded: {config.model.control.diagnose}")
        
        # Now test RefValue syntax
        refvalue_yaml = flexible_yaml.replace(
            "tstep: 300\n    diagnose: 0", 
            "tstep: {value: 600}\n    diagnose: {value: 1}"
        )
        yaml_file.write_text(refvalue_yaml)
        
        print("\nLoading configuration with RefValue syntax...")
        config2 = SUEWSConfig.from_yaml(str(yaml_file))
        tstep = config2.model.control.tstep
        diagnose = config2.model.control.diagnose
        
        # Check if values are accessible
        tstep_val = tstep.value if hasattr(tstep, 'value') else tstep
        diagnose_val = diagnose.value if hasattr(diagnose, 'value') else diagnose
        
        print(f"✅ tstep loaded: {tstep_val}")
        print(f"✅ diagnose loaded: {diagnose_val}")
        
        print("\n✅ Both syntaxes work correctly!")
        print("\nBenefits of flexible syntax:")
        print("- Consistency across all configuration fields")
        print("- Ability to add references/documentation to any field")
        print("- Backward compatibility maintained")
        
    except Exception as e:
        print(f"❌ Error: {e}")

## Summary

This notebook tests the complete solution for issue #543:

### Key Points:

1. **Default Behavior**: 
   - No automatic file generation
   - Clear instructions with full paths
   - Shows exact command to run

2. **Optional Automatic Generation**:
   - Enable with `auto_generate_annotated=True`
   - File is created immediately when validation issues found
   - Different message shown to users

3. **No Issues = No File**:
   - Even with `auto_generate_annotated=True`, no file is created if there are no validation issues

4. **SUEWSSimulation Compatibility**:
   - Uses default behavior (no auto-generation)
   - Users can still manually generate via config object

### Usage Examples:

```python
# Default (manual)
config = SUEWSConfig.from_yaml('config.yml')
# Then: config.generate_annotated_yaml('config.yml')

# Automatic
config = SUEWSConfig.from_yaml('config.yml', auto_generate_annotated=True)
# File created automatically if issues found
```